In [3]:
import os 
import glob
import sys
import eppy
import pandas as pd
from eppy.results import readhtml # the eppy module with functions to read the html
import pprint

In [35]:

pp = pprint.PrettyPrinter()

fname = "/Users/jmccarty/Desktop/RefBldgMediumOfficePre1980_V22-2-0_8A_USA_AK_FAIRBANKS-table.htm"
with open(fname, 'r') as html_doc:
    html_doc = html_doc.read()
    htables = readhtml.titletable(html_doc) # reads the tables with their titles


In [36]:
table_dict = {}

for table in htables:
    table_name = table[0]
    table_dict[table[0]] = pd.DataFrame(table[1][1:],columns=table[1][0])


In [37]:
table_dict['Comfort and Setpoint Not Met Summary']

,,Facility [Hours]
0,Time Setpoint Not Met During Occupied Heating,103.00
1,Time Setpoint Not Met During Occupied Cooling,20.83
2,Time Not Comfortable Based on Simple ASHRAE 55...,3631.50


In [ ]:
tables_of_interest = ['Discomfort-weighted Exceedance OccupiedHours', 
                      'Comfort and Setpoint Not Met Summary']